In [ ]:
#1) Missing values / Empty cells 
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("Sales.csv")

# Display original data
print("\n📋 Original Dataset:\n")
print(df)

# Clean date format preliminarily for easier handling later
df['Order Date'] = df['Order Date'].astype(str).str.replace("'", "", regex=False)

# Convert numeric columns (they may have NaNs as strings)
numeric_cols = ['Quantity', 'Unit Price', 'Total Revenue']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Handle missing values:

# For Customer Name, fill with "Unknown"
df['Customer Name'].fillna("Unknown", inplace=True)

# For Quantity, fill missing with median (most typical quantity)
df['Quantity'].fillna(df['Quantity'].median(), inplace=True)

# For Unit Price, fill missing with mean (average price)
df['Unit Price'].fillna(df['Unit Price'].mean(), inplace=True)

# For Total Revenue, if missing, calculate as Quantity * Unit Price
df['Total Revenue'] = df.apply(
    lambda row: row['Quantity'] * row['Unit Price'] if pd.isna(row['Total Revenue']) else row['Total Revenue'], axis=1)

# Display cleaned data after handling missing values
pd.set_option('display.max_rows', None)
print("\n✅ Dataset After Handling Missing Values:\n")
print(df)



📋 Original Dataset:

   Order ID  Customer Name   Order Date   Product  Quantity  Unit Price  \
0      1001       John Doe   01/01/2024  Widget A      10.0        25.0   
1      1002     Jane Smith   01/02/2024  Widget B       5.0        40.0   
2      1003            NaN  2024/01/03'  Widget A       NaN        25.0   
3      1004  Alice Johnson   04/01/2024  Widget C       3.0         NaN   
4      1005      Bob Brown  2024/01/05'  Widget B      10.0        40.0   
5      1006       John Doe   06/01/2024  Widget A       4.0        25.0   
6      1001       John Doe   01/01/2024  Widget A      10.0        25.0   
7      1007     Jane Smith   07/01/2024  Widget C      -6.0        70.0   

   Total Revenue  
0          250.0  
1          200.0  
2            NaN  
3          210.0  
4          400.0  
5          100.0  
6          250.0  
7         -420.0  

✅ Dataset After Handling Missing Values:

   Order ID  Customer Name  Order Date   Product  Quantity  Unit Price  \
0      1001   

C:\Users\DXB GADGETS\AppData\Local\Temp\ipykernel_12188\4091431052.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Customer Name'].fillna("Unknown", inplace=True)
C:\Users\DXB GADGETS\AppData\Local\Temp\ipykernel_12188\4091431052.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as

In [ ]:
#2) Inconsistent date formats  
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("Sales.csv")

# Step 1: Detect inconsistent date formats

# Clean apostrophes and spaces first for consistency in display
df['Order Date Cleaned'] = df['Order Date'].astype(str).str.replace("'", "", regex=False).str.strip()

# Try parsing dates normally
parsed_dates = pd.to_datetime(df['Order Date Cleaned'], errors='coerce', infer_datetime_format=True, dayfirst=False)

# Rows where parsing failed
inconsistent_dates = df[parsed_dates.isna()]

print("\n⚠️ Rows with Inconsistent or Unparseable Date Formats:\n")
print(inconsistent_dates[['Order Date']])

# Step 2: Fix dates

# First replace 'Order Date' with cleaned strings
df['Order Date'] = df['Order Date Cleaned']

# Try parsing dates again with dayfirst=True where it was previously NaT
parsed_dates = pd.to_datetime(df['Order Date'], errors='coerce', infer_datetime_format=True, dayfirst=False)
mask = parsed_dates.isna()
parsed_dates.loc[mask] = pd.to_datetime(df.loc[mask, 'Order Date'], errors='coerce', dayfirst=True)

# Fill missing dates by forward fill
parsed_dates.fillna(method='ffill', inplace=True)

# Update dataframe with fixed and uniform dates
df['Order Date'] = parsed_dates.dt.strftime('%Y-%m-%d')

# Drop helper column
df.drop(columns=['Order Date Cleaned'], inplace=True)

# Display cleaned dataset
pd.set_option('display.max_rows', None)
print("\n✅ Dataset After Cleaning Date Formats:\n")
print(df)



⚠️ Rows with Inconsistent or Unparseable Date Formats:

    Order Date
2  2024/01/03'
4  2024/01/05'

✅ Dataset After Cleaning Date Formats:

   Order ID  Customer Name  Order Date   Product  Quantity  Unit Price  \
0      1001       John Doe  2024-01-01  Widget A      10.0        25.0   
1      1002     Jane Smith  2024-01-02  Widget B       5.0        40.0   
2      1003            NaN  2024-03-01  Widget A       NaN        25.0   
3      1004  Alice Johnson  2024-04-01  Widget C       3.0         NaN   
4      1005      Bob Brown  2024-05-01  Widget B      10.0        40.0   
5      1006       John Doe  2024-06-01  Widget A       4.0        25.0   
6      1001       John Doe  2024-01-01  Widget A      10.0        25.0   
7      1007     Jane Smith  2024-07-01  Widget C      -6.0        70.0   

   Total Revenue  
0          250.0  
1          200.0  
2            NaN  
3          210.0  
4          400.0  
5          100.0  
6          250.0  
7         -420.0  


C:\Users\DXB GADGETS\AppData\Local\Temp\ipykernel_12188\2526693193.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed_dates = pd.to_datetime(df['Order Date Cleaned'], errors='coerce', infer_datetime_format=True, dayfirst=False)
C:\Users\DXB GADGETS\AppData\Local\Temp\ipykernel_12188\2526693193.py:27: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed_dates = pd.to_datetime(df['Order Date'], errors='coerce', infer_datetime_format=True, dayfirst=False)
C:\Users\DXB GADGETS\AppData\Local\Temp\ipykernel_12188\2526693193.py:32: FutureWarning: Series.

In [ ]:
#3) Duplicate rows.
import pandas as pd

# Load dataset
df = pd.read_csv("Sales.csv")

# Step 1: Detect duplicate rows (considering all columns)
duplicates = df[df.duplicated(keep=False)]  # show all duplicates, including original rows

print("\n🔍 Duplicate Rows Detected (including original duplicates):\n")
print(duplicates)

# Step 2: Remove duplicate rows keeping the first occurrence
df_cleaned = df.drop_duplicates()

# Step 3: Display dataset after removing duplicates
pd.set_option('display.max_rows', None)
print("\n✅ Dataset After Removing Duplicate Rows:\n")
print(df_cleaned)



🔍 Duplicate Rows Detected (including original duplicates):

   Order ID Customer Name  Order Date   Product  Quantity  Unit Price  \
0      1001      John Doe  01/01/2024  Widget A      10.0        25.0   
6      1001      John Doe  01/01/2024  Widget A      10.0        25.0   

   Total Revenue  
0          250.0  
6          250.0  

✅ Dataset After Removing Duplicate Rows:

   Order ID  Customer Name   Order Date   Product  Quantity  Unit Price  \
0      1001       John Doe   01/01/2024  Widget A      10.0        25.0   
1      1002     Jane Smith   01/02/2024  Widget B       5.0        40.0   
2      1003            NaN  2024/01/03'  Widget A       NaN        25.0   
3      1004  Alice Johnson   04/01/2024  Widget C       3.0         NaN   
4      1005      Bob Brown  2024/01/05'  Widget B      10.0        40.0   
5      1006       John Doe   06/01/2024  Widget A       4.0        25.0   
7      1007     Jane Smith   07/01/2024  Widget C      -6.0        70.0   

   Total Revenue  

In [ ]:
#4) Wrong data 
import pandas as pd

# Load dataset
df = pd.read_csv("Sales.csv")

# Clean 'Order Date' for consistency (optional but recommended)
df['Order Date'] = df['Order Date'].astype(str).str.replace("'", "", regex=False).str.strip()

# Convert numeric columns to numeric dtype for safe comparison
numeric_cols = ['Quantity', 'Unit Price', 'Total Revenue']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Identify wrong data conditions
wrong_quantity = df[df['Quantity'] <= 0]
wrong_unit_price = df[df['Unit Price'] <= 0]
wrong_revenue = df[df['Total Revenue'] <= 0]

# Check inconsistency between Total Revenue and Quantity * Unit Price (allowing small rounding difference)
revenue_mismatch = df[~((df['Total Revenue'] - (df['Quantity'] * df['Unit Price'])).abs() < 0.01)]

# Combine all wrong data rows (using index to avoid duplicates)
wrong_data = pd.concat([wrong_quantity, wrong_unit_price, wrong_revenue, revenue_mismatch]).drop_duplicates()

print("\n⚠️ Rows with Wrong Data Detected:\n")
print(wrong_data)

# Remove wrong data rows from original dataframe
df_cleaned = df.drop(index=wrong_data.index)

# Display cleaned dataset
pd.set_option('display.max_rows', None)
print("\n✅ Dataset After Removing Wrong Data:\n")
print(df_cleaned)



⚠️ Rows with Wrong Data Detected:

   Order ID  Customer Name  Order Date   Product  Quantity  Unit Price  \
7      1007     Jane Smith  07/01/2024  Widget C      -6.0        70.0   
2      1003            NaN  2024/01/03  Widget A       NaN        25.0   
3      1004  Alice Johnson  04/01/2024  Widget C       3.0         NaN   

   Total Revenue  
7         -420.0  
2            NaN  
3          210.0  

✅ Dataset After Removing Wrong Data:

   Order ID Customer Name  Order Date   Product  Quantity  Unit Price  \
0      1001      John Doe  01/01/2024  Widget A      10.0        25.0   
1      1002    Jane Smith  01/02/2024  Widget B       5.0        40.0   
4      1005     Bob Brown  2024/01/05  Widget B      10.0        40.0   
5      1006      John Doe  06/01/2024  Widget A       4.0        25.0   
6      1001      John Doe  01/01/2024  Widget A      10.0        25.0   

   Total Revenue  
0          250.0  
1          200.0  
4          400.0  
5          100.0  
6          250.0  

In [ ]:
#5) Unnecessary columns that are not relevant to the analysis. 
import pandas as pd

# Load dataset
df = pd.read_csv("Sales.csv")

# Step 1: Show original columns
print("\n📋 Original Columns in Dataset:\n")
print(df.columns.tolist())

# Step 2: Define relevant columns
relevant_cols = ['Order ID', 'Customer Name', 'Order Date', 'Product', 'Quantity', 'Unit Price', 'Total Revenue']

# Step 3: Identify unnecessary columns (not in relevant_cols)
unnecessary_cols = [col for col in df.columns if col not in relevant_cols]

print("\n🗑️ Unnecessary Columns to Drop:\n")
print(unnecessary_cols if unnecessary_cols else "No unnecessary columns found.")

# Step 4: Drop unnecessary columns if any
df_cleaned = df.drop(columns=unnecessary_cols) if unnecessary_cols else df.copy()

# Step 5: Display cleaned dataset
pd.set_option('display.max_rows', None)
print("\n✅ Dataset After Dropping Unnecessary Columns:\n")
print(df_cleaned)



📋 Original Columns in Dataset:

['Order ID', 'Customer Name', 'Order Date', 'Product', 'Quantity', 'Unit Price', 'Total Revenue']

🗑️ Unnecessary Columns to Drop:

No unnecessary columns found.

✅ Dataset After Dropping Unnecessary Columns:

   Order ID  Customer Name   Order Date   Product  Quantity  Unit Price  \
0      1001       John Doe   01/01/2024  Widget A      10.0        25.0   
1      1002     Jane Smith   01/02/2024  Widget B       5.0        40.0   
2      1003            NaN  2024/01/03'  Widget A       NaN        25.0   
3      1004  Alice Johnson   04/01/2024  Widget C       3.0         NaN   
4      1005      Bob Brown  2024/01/05'  Widget B      10.0        40.0   
5      1006       John Doe   06/01/2024  Widget A       4.0        25.0   
6      1001       John Doe   01/01/2024  Widget A      10.0        25.0   
7      1007     Jane Smith   07/01/2024  Widget C      -6.0        70.0   

   Total Revenue  
0          250.0  
1          200.0  
2            NaN  
3    